<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="cognitiveclass.ai logo">
</center>


# Machine Learning Foundation

## Course 4, Part e: Non-Negative Matrix Factorization DEMO


This exercise illustrates usage of Non-negative Matrix factorization and covers techniques related to sparse matrices and some basic work with Natural Langauge Processing.  We will use NMF to look at the top words for given topics.


## Data


We'll be using the BBC dataset. These are articles collected from 5 different topics, with the data pre-processed. 

These data are available in the data folder (or online [here](http://mlg.ucd.ie/files/datasets/bbc.zip?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01)). The data consists of a few files. The steps we'll be following are:

* *bbc.terms* is just a list of words 
* *bbc.docs* is a list of artcles listed by topic.

At a high level, we're going to 

1. Turn the `bbc.mtx` file into a sparse matrix (a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) format can be useful for matrices with many values that are 0, and save space by storing the position and values of non-zero elements).
1. Decompose that sparse matrix using NMF.
1. Use the resulting components of NMF to analyze the topics that result.


## Data Setup


Note: This lab has been updated to work in skillsnetwork for your convenience.


In [1]:
import urllib

In [2]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.mtx') as r:
    content = r.readlines()[2:]

## Part 1

Here, we will turn this into a list of tuples representing a [sparse matrix](https://docs.scipy.org/doc/scipy/reference/sparse.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01). Remember the description of the file from above:

* *bbc.mtx* is a list: first column is **wordID**, second is **articleID** and the third is the number of times that word appeared in that article.

So, if word 1 appears in article 3, 2 times, one element of our list will be:

`(1, 3, 2)`


In [3]:
sparsemat = [tuple(map(int,map(float,c.split()))) for c in content]
# Let's examine the first few elements
sparsemat[:8]

[(1, 1, 1),
 (1, 7, 2),
 (1, 11, 1),
 (1, 14, 1),
 (1, 15, 2),
 (1, 19, 2),
 (1, 21, 1),
 (1, 29, 1)]

## Part 2: Preparing Sparse Matrix data for NMF 


We will use the [coo matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) function to turn the sparse matrix into an array. 


In [4]:
import numpy as np
from scipy.sparse import coo_matrix
cols = [x[0] - 1 for x in sparsemat]
rows = [x[1] - 1  for x in sparsemat]
values = [x[2] for x in sparsemat]
coo = coo_matrix((values, (rows, cols)))
coo.toarray().shape

(2225, 9635)

## NMF


NMF is a way of decomposing a matrix of documents and words so that one of the matrices can be interpreted as the "loadings" or "weights" of each word on a topic. 


Check out [the NMF documentation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) and the [examples of topic extraction using NMF and LDA](http://scikit-learn.org/0.18/auto_examples/applications/topics_extraction_with_nmf_lda.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01).


## Part 3

Here, we will import `NMF`, define a model object with 5 components, and `fit_transform` the data created above.


In [5]:
# Surpress warnings from using older version of sklearn:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=818)
doc_topic = model.fit_transform(coo)

doc_topic.shape
# we should have 9636 observations (articles) and five latent features

(2225, 5)

In [6]:
# find feature with highest value per doc
np.argmax(doc_topic, axis=1)

array([1, 1, 1, ..., 3, 3, 2], dtype=int64)

## Part 4: 

Check out the `components` of this model:


In [7]:
model.components_.shape

(5, 9635)

This is five rows, each of which is a "topic" containing the weights of each word on that topic. The exercise is to _get a list of the top 10 words for each topic_. We can just store this in a list of lists.


**Note:** Just like we read in the data above, we'll have to read in the words from the `bbc.terms` file.


In [8]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.terms') as r:
    content = r.readlines()
words = [str(c.strip(), 'utf-8') for c in content]

In [9]:
import pandas as pd
topic_words = pd.DataFrame(model.components_, columns=words, index=['topic_'+str(i + 1) for i in range(5)])
topic_words

,ad,sale,boost,time,warner,profit,quarterli,media,giant,jump,...,£339,denialofservic,ddo,seagrav,bot,wirelessli,streamcast,peripher,headphon,flavour
topic_1,1.109794,0.000000,0.147521,1.805085,0.000000,0.000000,0.000000,0.290339,0.000000,0.007622,...,0.000000,0.000189,0.003569,0.005304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
topic_2,0.906610,2.135102,0.555638,1.619654,0.015276,1.232641,0.091248,0.058898,0.346788,0.206330,...,0.001106,0.000000,0.000000,0.000000,0.000000,0.000000,0.002087,0.002166,0.000000,0.001380
topic_3,0.678384,0.411546,0.050976,4.003071,0.037892,0.000000,0.000000,0.189766,0.084304,0.128306,...,0.000554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002014,0.004688,0.000000
topic_4,0.696425,0.415502,0.054580,1.219848,0.051868,0.051111,0.008216,1.221179,0.279640,0.000000,...,0.025132,0.019692,0.061144,0.074941,0.136037,0.020642,0.039628,0.029288,0.027964,0.020455
topic_5,0.475060,0.372709,0.123657,0.986447,0.021659,0.041599,0.000000,0.051945,0.020607,0.251904,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004888,0.000000,0.000000,0.000000


The original data had 5 topics, as listed in `bbc.docs` (which these topic words relate to). 

```
Business
Entertainment
Politics
Sport
Tech
```

In "real life", we would have found a way to use these to inform the model. But for this little demo, we can just compare the recovered topics to the original ones. And they seem to match reasonably well. The order is different, which is to be expected in this kind of model.


In [10]:
with urllib.request.urlopen('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/data/bbc.docs') as r:
    doc_content = r.readlines()
    
len(doc_content)

2225

In [11]:
topic_doc = pd.DataFrame(doc_topic, index=[str(i, 'utf-8').split('.')[0] for i in doc_content], columns=['topic_'+str(i+1) for i in range(5)])
topic_doc.reset_index().groupby('index').mean()

,topic_1,topic_2,topic_3,topic_4,topic_5
index,,,,,
business,0.023584,0.251565,0.007547,0.024401,0.006144
entertainment,0.020826,0.057755,0.036762,0.039328,0.154993
politics,0.261662,0.078559,0.014332,0.025091,0.009294
sport,0.026781,0.045077,0.167854,0.005876,0.025784
tech,0.023849,0.057403,0.115039,0.327589,0.020415


In [12]:
mapping = topic_doc.reset_index().groupby('index').mean().idxmax().to_frame().rename(columns={0:'Topics'})
mapping

,Topics
topic_1,politics
topic_2,business
topic_3,sport
topic_4,tech
topic_5,entertainment


In [13]:
result = pd.concat([mapping, topic_words], join='inner', axis=1)
result.set_index('Topics', inplace=True)

In [14]:
for i, topic in enumerate(result.index):
    display(result.sort_values(by=topic, axis=1, ascending=False).iloc[i, :].to_frame().T)

,parti,labour,govern,elect,blair,peopl,tori,minist,plan,brown,...,branson,slew,cowboi,bright,£26m,zheng,120m,lt,costeffect,flavour
politics,7.272066,6.859606,6.4028,5.984097,5.031092,4.831863,4.167926,4.078225,3.491092,3.330722,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,year,increas,wage,compani,busi,minimum,govern,rate,market,economi,...,insan,flak,vega,rene,gp,pager,xrai,pander,28th,bang
business,6.709741,4.237554,3.782039,3.15333,2.94221,2.915635,2.868437,2.629689,2.598373,2.466916,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,game,plai,time,player,world,first,win,get,go,on,...,bt,vendor,257,desktop,ibm,stiffen,lenovo,surplus,restraint,flavour
sport,15.865935,6.752707,4.003071,2.932378,2.871058,2.740374,2.613089,2.533307,2.487597,2.371216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,peopl,mobil,phone,technolog,servic,music,digit,user,network,on,...,madam,lahor,auf,strauss,jeanpierr,acclaim,turin,philharmon,jule,hinder
tech,6.515048,6.114551,5.233037,4.633311,3.961518,3.57757,2.946419,2.818726,2.634326,2.546418,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,best,song,music,year,award,film,25,angel,robbi,british,...,limp,scan,builtin,text,tablet,dock,laptop,antiviru,firewal,flavour
entertainment,11.895329,9.829428,9.578887,6.745143,6.661391,4.356695,4.104535,3.95287,3.34941,3.209357,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---
### Machine Learning Foundation (C) 2020 IBM Corporation
